In [98]:
import pandas as pd
from scipy import stats
import datetime
import rpy2
import seaborn as sns
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [165]:
df = pd.read_csv("ecs_series_1_uid_encoded(1).csv")

In [166]:
datetime_object = datetime.datetime.strptime("9/27/18 16:00", "%m/%d/%y %H:%M")
df['Date'] = df["ds"].apply(lambda x:datetime.datetime.strptime(x, "%m/%d/%y %H:%M"))
df = df.drop(columns = ['ds','region_no_factor','iz_no_factor','product_factor','instance_type_name_factor','instance_type_family_factor'])

In [167]:
df.sample(5)

,vcpu_net_delead,uid,Date
137011,0.066667,90605,2018-05-08 16:00:00
258528,0.021053,23446,2018-08-17 16:00:00
11463,0.033333,107366,2018-06-28 16:00:00
58635,-0.066667,103884,2018-07-16 16:00:00
184862,0.033333,73998,2018-03-28 16:00:00


In [168]:
df = df.groupby(by=['uid','Date']).sum().reset_index()
df = df.pivot(columns = 'Date',index = 'uid',values = 'vcpu_net_delead')

In [169]:
df_test = df.loc[3:8,:]

In [170]:
df_test

Date,2017-01-01 16:00:00,2017-01-02 16:00:00,2017-01-03 16:00:00,2017-01-04 16:00:00,2017-01-05 16:00:00,2017-01-06 16:00:00,2017-01-07 16:00:00,2017-01-08 16:00:00,2017-01-09 16:00:00,2017-01-10 16:00:00,...,2018-11-16 16:00:00,2018-11-17 16:00:00,2018-11-18 16:00:00,2018-11-19 16:00:00,2018-11-20 16:00:00,2018-11-21 16:00:00,2018-11-22 16:00:00,2018-11-23 16:00:00,2018-11-24 16:00:00,2018-11-25 16:00:00
uid,,,,,,,,,,,,,,,,,,,,,
3,0.044444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.044444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0.088889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,0.022222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,0.111111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,0.177778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,0.022222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Theta method

In [171]:
from rpy2.robjects.packages import importr
# import R's "base" package
# import rpy2's package module
import rpy2.robjects.packages as rpackages
import rpy2.robjects as robjects
# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list
# R package names
packnames = ['forecast']
# R vector of strings
from rpy2.robjects.vectors import StrVector
utils.install_packages(StrVector('forecast'))
base = importr('forecast')


In [190]:
# Theta just skips nan
[0 if (np.isnan(x)) else x for x in df_test.loc[3,:]]

[0.04444444400000001,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.177777778,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.088888889,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -0.177777778,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,

In [191]:
res = robjects.FloatVector([0 if (np.isnan(x)) else x for x in df_test.loc[3,:]])
rfun = base.thetaf(res,4)

In [192]:
#rfun - predictions
print(rfun)

    Point Forecast       Lo 80      Hi 80       Lo 95      Hi 95

694  -0.0006971361 -0.01440074 0.01300646 -0.02165498 0.02026071

695  -0.0006983685 -0.01440197 0.01300523 -0.02165622 0.02025948

696  -0.0006996008 -0.01440320 0.01300400 -0.02165745 0.02025825

697  -0.0007008331 -0.01440443 0.01300277 -0.02165868 0.02025701



In [177]:
rfun[1]

7.482091,7.964234,8.446377,8.928520
